In [1]:
!pip install simpletransformers
!pip install sentencepiece
!pip install simpletransformers
!pip install scipy
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=466c21fa3981d81437a70e5aa16cc5b26887ddd9ec99f43734fee6790fa0bf8e
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
Successfully built sklearn


In [2]:
import torch
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [3]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [4]:
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [5]:
#def get_response(df):
  

In [6]:
num_beams = 1
num_return_sequences = 1
#context = "The ultimate test of your knowledge is your capacity to convey it to another."
context = "\n\nThe bill was introduced in the US Senate on Wednesday, and the US House of Representatives got a vote on it on Thursday afternoon. The US President Donald Trump is expected to sign it. [continues...]"
get_response(context,num_return_sequences,num_beams)


['The bill was approved by the US House of Representatives on Thursday and is expected to be signed by the US President.']

In [7]:
from google.colab import drive
import pandas as pd

In [8]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/full.csv')
#test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
a_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/test.csv')
b_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/train.csv')


In [10]:
#drive.mount('/content/drive')
ptest_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased_test.csv')
ptrain_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased_train.csv')
pfull_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Fine-Tuning/Data/cs577/paraphrased.csv')

In [11]:
def get_hidden_states(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  #tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return translated

In [12]:
num_beams = 1
num_return_sequences = 1
row = df.iloc[0]
input_text = row['Text']
#print(row['Text'].type)
get_hidden_states(row['Text'], num_return_sequences,num_beams)

tensor([[    0,   110, 69098,   146,   656,   118,   200,   122,  6140,   112,
          6568,  2775,   113,  4585, 44078,   107,     1]], device='cuda:0')

In [13]:
batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt")["input_ids"].to(torch_device)
input_ids = tokenizer([input_text], return_tensors='pt')['input_ids'].to(torch_device)
#print(input_ids)
outputs = model(decoder_input_ids=input_ids, input_ids=input_ids, output_hidden_states=True, output_attentions=True, return_dict=True)
#print(outputs.keys())
print(len(outputs['encoder_hidden_states']))
#decoder_hidden_states = model(decoder_input_ids=input_ids, input_ids=input_ids, decoder_hidden_states=True)
#print(decoder_hidden_states)
#model(**batch)

17


In [14]:
hidden_state_1 = outputs['encoder_hidden_states'][0]
hidden_state_2 = outputs['encoder_hidden_states'][1]
hidden_state_3 = outputs['encoder_hidden_states'][2]
hidden_state_4 = outputs['encoder_hidden_states'][3]
print(hidden_state_1)

tensor([[[-9.0761e+00,  1.5051e-02,  1.8805e+01,  ...,  9.6768e+00,
          -1.1028e+00, -1.5832e+01],
         [-1.2719e+01,  1.1861e+00, -1.0433e+01,  ...,  8.0127e-01,
           1.1446e+01, -1.7039e+01],
         [ 4.2427e+00,  3.1670e+00, -4.7095e+00,  ..., -2.4411e+01,
           2.9047e+00,  8.9822e+00],
         ...,
         [ 5.6438e+00,  1.8921e+00, -5.1129e+00,  ..., -1.3354e+01,
           1.7963e+01,  6.2575e+00],
         [-5.7622e+00,  2.9635e-01,  3.0068e+01,  ...,  1.8978e+01,
          -1.0643e+01, -7.8753e+00],
         [-1.9474e+01, -6.8590e-01,  5.5070e+01,  ..., -3.7550e+00,
           1.2482e+01, -2.5112e+00]]], device='cuda:0', grad_fn=<AddBackward0>)


In [15]:
arr = hidden_state_1.cpu().detach().numpy()
arr[0].size
hidden_state_1.size()

torch.Size([1, 37, 1024])

In [16]:
# squeeze hidden states to [1, 37]
# multiple by batch and average the values
# take distance between prob distributions
batch.size()

torch.Size([1, 37])

In [17]:
torch.squeeze(hidden_state_1).size()

torch.Size([37, 1024])

In [18]:
print(outputs.keys())

odict_keys(['logits', 'past_key_values', 'decoder_hidden_states', 'decoder_attentions', 'cross_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])


In [19]:
outputs["encoder_last_hidden_state"]

tensor([[[-0.2803,  0.0013, -0.0394,  ..., -0.1274, -0.0186, -0.1597],
         [ 0.0430, -0.0340, -0.0628,  ..., -0.0191,  0.1115, -0.0469],
         [-0.2939, -0.0565, -0.0238,  ...,  0.1170, -0.0710, -0.0403],
         ...,
         [ 0.0513, -0.0866,  0.0062,  ...,  0.0477, -0.0542,  0.1495],
         [-0.0704,  0.1189,  0.0557,  ...,  0.3864, -0.1074,  0.1651],
         [ 0.0465, -0.1088,  0.0330,  ...,  0.0698,  0.0154,  0.0090]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [20]:
outputs["encoder_last_hidden_state"].size()

torch.Size([1, 37, 1024])

In [21]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Optional model configuration
config = {"output_hidden_states": True}
class_model_args = ClassificationArgs(num_train_epochs=1, config=config)
# Create a ClassificationModel
class_model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=4,
    args=class_model_args,
    use_cuda=False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [22]:
# Preparing eval data
eval_data = [
    ["Aragorn was the heir of Elendil", 1],
    ["Sam was the heir of Isildur", 0],
    ["Merrry is stronger than Pippin", 2],
]
eval_df = pd.DataFrame(eval_data)
outputs = class_model.predict(["Aragorn was the heir of Elendil"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
len(outputs)

4

In [24]:
outputs1 = class_model.predict(["Sam was the heir of Isildur"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
import numpy as np
last_hidden_state = outputs[3][11]
print(last_hidden_state.shape)
matrix = np.zeros((1,128,768))
print(matrix.shape)

(1, 128, 768)
(1, 128, 768)


In [26]:
auth0_df = pfull_df[pfull_df["Author"] == 0]
auth1_df = pfull_df[pfull_df["Author"] == 1]
auth2_df = pfull_df[pfull_df["Author"] == 2]
auth3_df = pfull_df[pfull_df["Author"] == 3]

In [27]:
auth0_df = auth0_df.head(10)
auth1_df = auth1_df.head(10)
auth2_df = auth2_df.head(10)
auth3_df = auth3_df.head(10)

In [28]:
for row in auth0_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth0_df)
avg_hidden_layer = np.divide(matrix, datapoints)
print(avg_hidden_layer)



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.43222218 -0.0343784  -0.08040118 ... -0.1937808   0.20505023
    0.00116059]
  [ 0.41906697 -0.48405848  0.41524624 ... -0.10736673  0.01738078
    0.1373135 ]
  [ 0.21510761 -0.04740868  0.08526777 ...  0.19564448 -0.10114786
   -0.02264913]
  ...
  [ 0.19655168  0.07655873 -0.01327539 ...  0.01707234  0.0850656
    0.15837855]
  [ 0.1394142   0.06662932  0.00623537 ... -0.01396589  0.06382897
    0.14528187]
  [ 0.14872779 -0.03968011 -0.01069056 ... -0.0499383   0.08566071
    0.07410865]]]


In [29]:
for row in auth1_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth1_df)
avg_hidden_layer1 = np.divide(matrix, datapoints)
print(avg_hidden_layer1)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.63020888 -0.02585093 -0.23868463 ... -0.29306219  0.52135604
    0.0968476 ]
  [ 0.67036928 -0.83553352  0.78067435 ... -0.1206894   0.09094621
    0.25863011]
  [ 0.21003955  0.05477016  0.08799112 ...  0.47297434  0.07909129
    0.06740898]
  ...
  [ 0.11341162  0.11639959 -0.02750497 ...  0.09065181  0.12598203
    0.30479317]
  [ 0.08778113  0.07327517 -0.0147375  ...  0.10070201  0.08853788
    0.22150478]
  [ 0.0510712  -0.03596327 -0.08985703 ...  0.06279527  0.12884128
    0.20988072]]]


In [30]:
for row in auth2_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth2_df)
avg_hidden_layer2 = np.divide(matrix, datapoints)
print(avg_hidden_layer2)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 0.98972307 -0.02159075 -0.32395109 ... -0.4499398   0.78792314
    0.15481589]
  [ 0.95664719 -1.24601024  1.11721698 ... -0.03676733  0.2360972
    0.29997599]
  [ 0.40455974 -0.08662268 -0.01812443 ...  0.83525584  0.14225215
    0.15171414]
  ...
  [ 0.25236399  0.11653293 -0.07920499 ...  0.11802018  0.13952083
    0.40246953]
  [ 0.1375301   0.05674576 -0.00388253 ...  0.16769244  0.27063007
    0.37127479]
  [ 0.06712765 -0.06949105 -0.11600642 ...  0.16091297  0.25414715
    0.37103545]]]


In [31]:
for row in auth3_df.iterrows():
  text = row[1]['Paraphrased Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(auth3_df)
avg_hidden_layer3 = np.divide(matrix, datapoints)
print(avg_hidden_layer3)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 1.33602426 -0.02044357 -0.42813124 ... -0.64496483  1.18153617
    0.24451964]
  [ 1.33775649 -1.70741232  1.54586672 ... -0.24643717  0.39905027
    0.445543  ]
  [ 0.69274648 -0.26323691 -0.08248219 ...  0.89243609  0.312405
    0.40110069]
  ...
  [ 0.31907545  0.05238474 -0.02277878 ...  0.13414911  0.22977316
    0.49558151]
  [ 0.20463159 -0.02216718  0.05138296 ...  0.21308132  0.40497961
    0.49884379]
  [ 0.100625   -0.1275812  -0.10835897 ...  0.18106508  0.40034481
    0.48951567]]]


In [32]:
from scipy.spatial import distance
def mahalanobis(P, Q):
    stacked_matrix = np.stack((P, Q), axis=0)
    covariance_matrix = np.cov(stacked_matrix)
    inverse_covariance_matrix = np.linalg.inv(covariance_matrix)
    return distance.mahalanobis(P, Q, inverse_covariance_matrix)

def bhattacharyya_distance(P, Q):
    STEPS = 100
    dP = gaussian_kde(P)
    cov_factor = 0.1
    dP.covariance_factor = lambda:cov_factor
    dP._compute_covariance()

    dQ = gaussian_kde(Q)
    cov_factor = 0.1
    dQ.covariance_factor = lambda:cov_factor
    dQ._compute_covariance()

    combinedProb = np.concatenate((P, Q))

    coef = np.linspace(np.min(combinedProb), np.max(combinedProb), STEPS)
    bhatt_coef = 0
    for c in coef:
        p = dP(c)
        q = dQ(c)
        bhatt_coef += np.sqrt(p*q) * (np.min(combinedProb), np.max(combinedProb)) / STEPS

    if bhatt_coef == 0:
        return float('Inf')
    
    return -np.log(bhatt_coef)

In [33]:
from sklearn.preprocessing import normalize
from scipy.special import expit
p0 = np.squeeze(avg_hidden_layer)
p1 = np.squeeze(avg_hidden_layer1)
p2 = np.squeeze(avg_hidden_layer2)
p3 = np.squeeze(avg_hidden_layer3)
normp0 = expit(p0)
normp1 = expit(p1)
normp2 = expit(p2) #normalize(p2, axis=0, norm='l1')
normp3 = expit(p3) # normalize(p3, axis=0, norm='l1')
dist0_1 = distance.jensenshannon(normp0, normp1)
dist0_2 = distance.jensenshannon(normp0, normp2)
dist0_3 = distance.jensenshannon(normp0, normp3)

dist1_2 = distance.jensenshannon(normp1, normp2)
dist1_3 = distance.jensenshannon(normp1, normp3)

dist2_3 = distance.jensenshannon(normp2, normp3)


In [34]:
print(np.average(dist1_3))
print(np.average(dist1_2))

0.027197188279399542
0.015759418965033994


In [35]:
print(np.average(dist0_1))
print(np.average(dist0_2))
print(np.average(dist0_3))
print(np.average(dist1_2))
print(np.average(dist1_3))
print(np.average(dist2_3))

0.01674355594666908
0.027188171112859868
0.03840446975013938
0.015759418965033994
0.027197188279399542
0.016584729427863584


In [36]:
avg_hidden_layer2

array([[[ 0.98972307, -0.02159075, -0.32395109, ..., -0.4499398 ,
          0.78792314,  0.15481589],
        [ 0.95664719, -1.24601024,  1.11721698, ..., -0.03676733,
          0.2360972 ,  0.29997599],
        [ 0.40455974, -0.08662268, -0.01812443, ...,  0.83525584,
          0.14225215,  0.15171414],
        ...,
        [ 0.25236399,  0.11653293, -0.07920499, ...,  0.11802018,
          0.13952083,  0.40246953],
        [ 0.1375301 ,  0.05674576, -0.00388253, ...,  0.16769244,
          0.27063007,  0.37127479],
        [ 0.06712765, -0.06949105, -0.11600642, ...,  0.16091297,
          0.25414715,  0.37103545]]])

In [40]:
df0 = df[df["Author"] == 0]
df1 = df[df["Author"] == 1]
df2 = df[df["Author"] == 2]
df3 = df[df["Author"] == 3]
df0 = df0.head(10)
df1 = df1.head(10)
df2 = df2.head(10)
df3 = df3.head(10)

In [41]:
for row in df0.iterrows():
  text = row[1]['Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(df0)
avg_hidden_layer0 = np.divide(matrix, datapoints)
print(avg_hidden_layer0)

for row in df1.iterrows():
  text = row[1]['Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(df1)
avg_hidden_layer1 = np.divide(matrix, datapoints)
print(avg_hidden_layer1)

for row in df2.iterrows():
  text = row[1]['Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(df2)
avg_hidden_layer2 = np.divide(matrix, datapoints)
print(avg_hidden_layer2)

for row in df3.iterrows():
  text = row[1]['Text']
  ls = []
  ls.append(text)
  outputs = class_model.predict(ls)
  last_hidden_state = outputs[3][11]
  matrix = np.add(matrix, last_hidden_state)

datapoints = len(df3)
avg_hidden_layer3 = np.divide(matrix, datapoints)
print(avg_hidden_layer3)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[[ 3.13497142  0.43740332 -1.00138796 ... -1.79980865  2.76597619
    0.87440756]
  [ 2.86135722 -3.26380403  3.20928376 ... -0.86951535  1.34733486
    1.12127325]
  [ 0.60183578 -0.40762257  0.16217341 ...  1.80752917  1.45553539
    1.15361193]
  ...
  [ 0.72849435  0.26041812 -0.18257362 ...  0.08045504  0.80266695
    0.90405642]
  [ 0.84115621  0.07850168 -0.23810783 ...  0.15608812  0.92692425
    0.92454827]
  [ 0.60100926 -0.15289516 -0.17974887 ...  0.10281328  0.96056721
    0.94905331]]]


In [42]:
from sklearn.preprocessing import normalize
from scipy.special import expit
p0 = np.squeeze(avg_hidden_layer)
p1 = np.squeeze(avg_hidden_layer1)
p2 = np.squeeze(avg_hidden_layer2)
p3 = np.squeeze(avg_hidden_layer3)
normp0 = expit(p0)
normp1 = expit(p1)
normp2 = expit(p2) #normalize(p2, axis=0, norm='l1')
normp3 = expit(p3) # normalize(p3, axis=0, norm='l1')
dist0_1 = distance.jensenshannon(normp0, normp1)
dist0_2 = distance.jensenshannon(normp0, normp2)
dist0_3 = distance.jensenshannon(normp0, normp3)

dist1_2 = distance.jensenshannon(normp1, normp2)
dist1_3 = distance.jensenshannon(normp1, normp3)

dist2_3 = distance.jensenshannon(normp2, normp3)

print(np.average(dist0_1))
print(np.average(dist0_2))
print(np.average(dist0_3))
print(np.average(dist1_2))
print(np.average(dist1_3))
print(np.average(dist2_3))

0.056811487236534675
0.06660859989274175
0.07516820513033474
0.022117528944362638
0.0325395438801778
0.019829230078523347
